## Importing the dataset and getting an overview

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pywaffle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 4.1 MB/s eta 0:00:00


In [ ]:
pip install geopandas

In [ ]:
pip install folium

In [ ]:
import pandas as pd #for working with dataframes
import matplotlib.pyplot as plt #for general visualizations
from pywaffle import Waffle #for waffle charts
import geopandas as gpd #for working with geographical data
import folium as fm #for visualizing with maps
from IPython.display import display

mammal_data = pd.read_excel(r"/content/drive/MyDrive/Python/arter_pattedyr.xlsx")

FileNotFoundError: ignored

The dataset we have chosen is from arter.dk, which is...

First of all, we describe som basic details about the dataset.

In [ ]:
print(f"The dataset contains {mammal_data.shape[0]} rows and {mammal_data.shape[1]} columns. Thereby there are {mammal_data.size} cells in the dataset.") #describing the numbers of rows, columns, and cells in the dataset

In [ ]:
print("The columns contain these types of data:\n")
mammal_data.info()

Next, we make sure that the license permits us to use the data.

In [ ]:
print(pd.isnull(mammal_data['License']).sum()) #confirming that there are no missing values in the license column
print(mammal_data.groupby('License')['License'].count()) #confirming that there are no differing license type for any data points

The CC by 4.0 means that we can use the data...

## Exploring and cleaning the dataset

Next, we remove all columns, where more data points are missing than not, as we only wish to use columns where we can use close to all rows for analysis.

In [ ]:
mammal_data.dropna(axis=1, thresh = 0.5 * len(mammal_data), inplace = True) #removing columns with missing values in more than half the observations
mammal_data.info() #displaying the remaining columns

In the next step, we found out that two of the columns contain personal information by having the name of the observer of the animal and the one adding it to the database. We believe that it is unethical to display PIIs unnessecarily, and therefore we drop those columns before displaying the content of the dataframe.

In [ ]:
#removing the columns with names of persons
mammal_data.drop(['Observatør(er)', #not relevant. Can perhaps be transformed into numerical values and be used to predict subregion (location)
                  'Indtaster'], #not relevant
                  axis=1, inplace = True)

Now we are able to get a better overview of what the remaining columns actually contain.

In [ ]:
pd.set_option('display.max_columns', None) #displaying all columns instead of just some
mammal_data.head(10) #displaying the 10 first rows

Upon inspection, we remove some of the columns. The reason is stated in a comment.

In [ ]:
mammal_data.drop(['Systemoprindelse', #not relevant
                  'Observationstidspunkt', #too specific to be useful
                  'Indsamlet',#not relevant
                  'Fundtype', #not relevant
                  'Valideret', #not relevant
                  'License', #we already confirmed that all data points has the same license and that we can use the data
                  'Link', #not relevant
                  'Arter taxon ID', #not relevant
                  'Sløret', #not relevant
                  'Klasse latinsk navn', #redundant as the chosen data only contains mammals
                  'Klasse dansk navn', #redundant as the chosen data only contains mammals
                  'Accepteret dansk art', #not relevant
                  'Artsgruppe'], #redundant as the chosen data only contains mammals
                  axis=1, inplace = True)

Now let's have a look at the remaining data.

In [ ]:
mammal_data.head(10) #displaying the 10 first rows

Now we would like to convert the observation date to year, as we only wish to visualize and categorize per year.

In [ ]:
mammal_data["Observationsdato"] = mammal_data["Observationsdato"].str.split("-").str[1] #only keeping the characters after the hyphen in the date

We want to inspect the different types of data in the 'Taxonrang' column, as there seems to be some rare values. Ideally 'Taxonrang' should only include 'Art', meaning species.

In [ ]:
mammal_data.groupby('Taxonrang')['Taxonrang'].count()

Then we inspect rows containing the three rarest values to see why they differ.


In [ ]:
mammal_data.loc[mammal_data['Taxonrang'] == 'SLÆGT']

In [ ]:
mammal_data.loc[mammal_data['Taxonrang'] == 'SUPERART']

For 'SUPERART' and 'SLÆGT', it is related to two types of bats being near identical in looks, why they were formerly considered the same species. Therefore the observer has been in doubt about the exact species. For convenience, we will convert all these into 'Pipistrelflagermus' in the column 'Taxon dansk navn', as it would not make sense to have them separated when visualizing the data.



[About 'superarter'](https://om.arter.dk/vidensbase/hjaelp-til-arters-webside/artsbogen/der-findes-arter-men-ogsa-slaegter-ordener-og-taxa/)

[About the difference between Pipistrellus and Pygmaeus](https://pattedyratlas.lex.dk/Pipistrelflagermus)


In [ ]:
mammal_data.loc[[5442, 1439, 4408],'Taxon dansk navn'] = "Pipistrelflagermus" #changing the value manually based on the index

For 'UNDERART', 'Muflon' does not match another species directly, as it is just a more specific indicator, therefore we will leave it be.

In [ ]:
mammal_data.loc[mammal_data['Taxonrang'] == 'UNDERART']

It is in the same family as the 'Europæisk bison', but it would not be correct to convert it to the same.

In [ ]:
mammal_data.loc[mammal_data['Familie latinsk navn'] == 'Bovidae']

As we saw in the beginning, the Danish order name has many more missing values than the Lation order name, why we will focus on filling in that column for later use. In 245 cases, the observer has just put in the taxonimical order, likely if they have not been aware of the exact species.

In [ ]:
df_nan_orden = mammal_data[mammal_data["Orden latinsk navn"].isna()]
df_nan_orden.groupby("Taxon dansk navn")["Taxon dansk navn"].count()

We now know that in 244 cases, it has been an unspecified bat, and in 1 case it has been an unspecified whale. As the order is mentioned in the taxon coloumn, it is not also in the order column, but we would like to have it this way.

In [ ]:
row_selector_bat = mammal_data["Taxon dansk navn"] == "Flagermus"
mammal_data.loc[row_selector_bat, "Orden latinsk navn"] = "Chiroptera"

row_selector_whale = mammal_data["Taxon dansk navn"] == "Hvaler"
mammal_data.loc[row_selector_whale, "Orden latinsk navn"] = "Cetacea"

mammal_data["Taxon dansk navn"].isnull().values.any() #checks for any missing values in the column

Now there are no more missing values in the latin order column.

We could do the same check for the rows with the value 'Familie', meaning family, in 'Taxonrang', but we will use the order rather than the family for analysis, and therefore it is redundant to fill the missing values in that column as well.

After our initial cleaning, we will now use groupby and descriptive statistics to get to know our values better.

In [ ]:
mammal_data.groupby("Observationsdato")["Observationsdato"].count() #counting observations per year

Now we can see that nearly all observations of animals are between 2020 and 2023. This is important to consider, as we wish to do some graphs over time, and this is not a very long useful timespan.



As we do not memorize the latin family names by heart, we want to display the most common animal for each family to help get an overview.

In [ ]:
mammal_data.groupby("Orden latinsk navn")["Taxon dansk navn"].agg(pd.Series.mode)

Now we wish to see some basic descriptive statistics on the 'Orden latinsk navn' column

In [ ]:
mammal_data["Orden latinsk navn"].describe()

To see more detail in terms of the frequency of all unique values, we check the amount of animals in the different orders.

In [ ]:
mammal_data['Orden latinsk navn'].value_counts()

To visialize the proportion of each animal order we have used a waffle chart, which we find more easily interpretable than a pie chart. Due to the very limited amount of free-to-use icons available, we had to be a little creative on the visual representations.

In [ ]:
order_counts = mammal_data['Orden latinsk navn'].value_counts()
total_animals = order_counts.sum()

# Waffle chart
plt.figure(
    FigureClass = Waffle,
    rows = 11,
    values = order_counts / total_animals * 100,
    icons = ['cow', 'dog', 'carrot', 'mouse', 'baseball-bat-ball', 'road-spikes', 'person-digging', 'water', 'horse'],
    icon_legend = True,
    legend = {
        'labels': ['Artiodactyla', 'Carnivora', 'Lagomorpha', 'Rodentia', 'Chiroptera', 'Erinaceomorpha', 'Soricomorpha', 'Cetacea', 'Perissodactyla'],
        'loc': 'upper left',
        'bbox_to_anchor': (1, 1)})
print() #this prevents the chart from being displayed twice

Perissodactyla has a too small proportion of the data to be displayed here.

## Geographical data

One of the main reasons for choosing this dataset was because it includes coordinate data for the observations.

In [ ]:
#copying the relevant columns and adding them to a new df
map_columns = {'Latitude': mammal_data['Lat'].copy(),
               'Longitude': mammal_data['Long'].copy(),
               'Animal': mammal_data['Taxon dansk navn'].copy(),
               'Order': mammal_data['Orden latinsk navn'].copy(),
               'Inaccuracy (m)': mammal_data['Usikkerhed (m)'].copy(),
               'Year': mammal_data["Observationsdato"].copy()}
map_df = pd.DataFrame(map_columns)

As we want the map to be relatively accurate, we remove all observations where the uncertainty about the location is larger than 1500 meters. We also make sure to round off to less digits in the floats.

In [ ]:
map_df = map_df.drop(map_df[map_df['Inaccuracy (m)'] > 1500].index)
map_df['Inaccuracy (m)'] = map_df['Inaccuracy (m)'].round(2)

For the observations where inaccuracy was not specified, we fill in with "unknown" as it looks better than NaN for display on the map.

In [ ]:
map_df['Inaccuracy (m)'] = map_df['Inaccuracy (m)'].fillna("Unknown")

Based on geographical data on the coordinate range of the five Danish regions, we want to perform a spatial join to extract what region the observation ocurred and add it as a column in the dataframe. The data regarding Danish regions are from https://www.diva-gis.org/gdata

In [ ]:
denmark_regions = gpd.read_file("/content/drive/MyDrive/Python/Regioner/DNK_adm1.shp") #importing coordinates on the regions of Denmark - NB: all the DNK_adm1 files must be in the same folder

coordinates_gdf = gpd.GeoDataFrame(map_df, geometry=gpd.points_from_xy(map_df.Longitude, map_df.Latitude)) #extracting the coordinate set from the lat. and long.
coordinates_gdf = coordinates_gdf.set_crs("EPSG:4326") #matching the coordinate reference system of the GDFs based on information in the error message

spatial_join = gpd.sjoin(coordinates_gdf, denmark_regions, how='left', predicate='within') #matching the coordinates of the observations with the regions with spatial join

map_df['Region'] = spatial_join['NAME_1'] #adding the name of the region as a column in the df

We wish to investigate the reason for missing values in the region data, as all observations include coordinates. Therefore we create a subset only including rows with missing region value and display it.

In [ ]:
map_region_nan = map_df[map_df["Region"].isna()]
map_region_nan["Animal"].value_counts()

Now we can see that the majority of the missing values relates to mammals living in the sea like seals and whales, where otters and mink are known swimmers as well, which explains why they do not belong to a region. There are also a few bats that might have been flying over the sea. Though, there are also more unexpected values like deer, hares, foxes, and squirrels, which calls for a closer look at the accuracy of the region separation. As we deal with data entered by people, who can make errors, and we have accepted an inaccuracy of 1500 meters or less, it might relate to animals that was observed very close to the shore. We assume this for now and fill in the missing values with "In and by the ocean".

In [ ]:
map_df["Region"].fillna("I eller ved havet", inplace = True)
map_df["Region"].isnull().values.any() #checks for any missing values in the column

Now an overview of the amount of animal observations for each region.

In [ ]:
map_df["Region"].value_counts()

To investigate the locations of the observations, we create a map displaying pins for only the newly created region "I eller ved havet".

In [ ]:
sea_and_shore_df = map_df.loc[map_df['Region'] == "I eller ved havet"] #creating a df including only animals by the ocean
sea_and_shore_map = fm.Map(location=[56.2639, 9.5018], zoom_start=7) #creating a map of Denmark

#looping through the relevant information for each row in the df
for lat, lon, animal, order, region, inaccuracy, year in zip(sea_and_shore_df['Latitude'],
                                                             sea_and_shore_df['Longitude'],
                                                             sea_and_shore_df['Animal'],
                                                             sea_and_shore_df['Order'],
                                                             sea_and_shore_df['Region'],
                                                             sea_and_shore_df['Inaccuracy (m)'],
                                                             sea_and_shore_df["Year"]):
    popup_text = f"Animal: {animal}<br>Order: {order}<br>Region: {region}<br>Inaccuracy: {inaccuracy} meters<br>Year: {year}" #choosing the info for the pop up
    fm.Marker([lat, lon], popup=fm.Popup(popup_text, max_width=300)).add_to(sea_and_shore_map) #adding the pins to the map

display(sea_and_shore_map) #displaying the map with the pins

This visualization shows us that most of the animals are indeed observed in the ocean or in the fjords. Though, some of the pins seem to be placed on land very close to the shore rather than in water. Very few of the pins in the southernmost Jutland are not placed near the ocean, and they are most likely included because the region coordinate data considers them to be placed in Germany. This shows us that the unexpected observations of land animals outside of the five regions is likely related to the GIS data containing the regions being a bit inaccurate. This was important to explore to know how much we can trust the accuracy of our maps.

## Addressing the research question

As our research question is primarily interested in

In [ ]:
df_hunter_and_prey = map_df.drop(map_df[map_df['Region'] == "I eller ved havet"].index)
df_hunter_and_prey

In [ ]:
list(df_hunter_and_prey.loc[df_hunter_and_prey['Order'] == 'Carnivora', 'Animal'].unique())

predators_todrop = ['Grævling','Tamkat','Spættet sæl','Hvalros','Gråsæl','Ægte sæler','Odder','Katte']

# Filtering the DataFrame based on the condition
filtered_carnivora = df_hunter_and_prey[df_hunter_and_prey['Order'] == 'Carnivora']

# Dropping rows with specific values in the 'Animal' column
filtered_carnivora = filtered_carnivora[~filtered_carnivora['Animal'].isin(predators_todrop)]

# Now, if you want to see the unique values after dropping, you can use unique()
unique_values_after_drop = filtered_carnivora['Animal'].unique()

# If you want to modify the original DataFrame, you can use loc to update the values
df_hunter_and_prey.loc[df_hunter_and_prey['Order'] == 'Carnivora', 'Animal'] = filtered_df['Animal']




## Tanker og noter
- Vise sammenhæng mellem ræv og rævemad (gnavere og støttetandede)
- Lave et nyt kort med kun ræve og byttedyr for se, om der er sammenhæng i lokationen. Forskellige farver på pins.
- Måske skal vi tage udgangspunkt i procent af antal observationer frem for antal
- Hvordan kan vi bruge desktiptiv statistik?
- Haakon siger, at vi skal være opmærksomme på hvilken slags graf vi bruger til hvilke formål
- [StackOverflow: Plotting three dimensions of categorical data in Python
](https://stackoverflow.com/questions/58303175/plotting-three-dimensions-of-categorical-data-in-python) - En nice visualisering vi kunne bruge med region, år og orden
- Hvad med Cramers V?
- Det er måske lidt for voldsomt med et kort med alle datapunkterne, da Folium er ret tungt, og det ikke rigtig tjener et formål
- Måske et subset til at tjekke dyrene i og ved havet
- Evt. bar charts med andel af byttedyr og ræve for hver region (5*2 søjler)
- Alternativt sådan en stablet søjle til evt. at vise bytte, ræv og "andet"

Byttedyr:
- Lagomorpha
- Rodentia
- Soricomorpha
- Erinaceomorpha

Rovdyr:
- Alle, der ikke lever i havet + grævling, odder, tamkat